<a href="https://colab.research.google.com/github/deepthi50997/machine-learning-/blob/main/Gesture%20Controlled%20Virtual%20Mouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opencv-python mediapipe pyautogui

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.2/171.2 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  

In [4]:
!python gesture_mouse.py

python3: can't open file '/content/gesture_mouse.py': [Errno 2] No such file or directory


In [6]:
import cv2
import mediapipe as mp
import pyautogui
import time
import numpy as np
from google.colab.patches import cv2_imshow

# Initialize MediaPipe and PyAutoGUI
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Get screen size (this might not work as expected in a headless environment)
# Consider if you actually need pyautogui's screen size for this application in Colab
screen_w, screen_h = pyautogui.size()


# Start webcam
cap = cv2.VideoCapture(0)
prev_click_time = 0

while True:
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get coordinates of index finger tip and thumb tip
            index_tip = hand_landmarks.landmark[8]
            thumb_tip = hand_landmarks.landmark[4]

            index_x = int(index_tip.x * w)
            index_y = int(index_tip.y * h)

            # Map hand position to screen coordinates
            # Adjust scaling and offset as needed for your use case in Colab
            screen_x = int(index_tip.x * screen_w)
            screen_y = int(index_tip.y * screen_h)

            # Move the mouse to index finger position (this might not function
            # as a typical mouse in the Colab environment)
            pyautogui.moveTo(screen_x, screen_y)

            # Distance between index and thumb to detect "click"
            distance = np.linalg.norm(
                np.array([index_x, index_y]) -
                np.array([int(thumb_tip.x * w), int(thumb_tip.y * h)])
            )

            if distance < 40:
                current_time = time.time()
                if current_time - prev_click_time > 0.5:
                    # Simulate a click (this might not function as a typical
                    # click in the Colab environment)
                    pyautogui.click()
                    prev_click_time = current_time
                    cv2.putText(frame, 'Click!', (index_x, index_y - 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.circle(frame, (index_x, index_y), 8, (255, 0, 255), -1)

    # Use cv2_imshow to display the frame in the Colab output
    cv2_imshow(frame)

    # Check for ESC key press (this might not work as expected with cv2_imshow)
    # You might need a different mechanism to stop the loop in Colab
    if cv2.waitKey(1) & 0xFF == 27:  # ESC key to stop
        break

cap.release()
cv2.destroyAllWindows()

KeyError: 'DISPLAY'